In [ ]:
# @title 挂载Google硬盘
%%capture
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title 控制显存碎片配置
import os, sys

# 必须在 import torch / diffusers / transformers 之前运行
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:128"

# 防呆：如果 torch 已经被导入了，就说明这格跑晚了（本次不会生效）
if "torch" in sys.modules:
    raise RuntimeError("torch 已导入：allocator 配置本次不会生效。请重启 runtime 后第一格先跑这个 cell。")

print("✅ PYTORCH_CUDA_ALLOC_CONF =", os.environ["PYTORCH_CUDA_ALLOC_CONF"])

In [ ]:
# @title 环境配置
%%capture
!pip install -U --prefer-binary -q \
  git+https://github.com/Disty0/sdnq \
  git+https://github.com/huggingface/diffusers

!pip install -U -q transformers accelerate

In [ ]:
# @title 🔥 安装 cache-dit（含量化与缓存加速支持）
%%capture
!pip install -U -q "cache-dit[quantization]"

In [ ]:
# @title 🤖 模型管理器（混搭版）
import ipywidgets as widgets
from IPython.display import display
import torch
import os
import gc

# ----------------------------
# ✅ TF32 精度控制 (PyTorch 新 API)
# ----------------------------
try:
    torch.set_float32_matmul_precision('high')
    torch.backends.cudnn.conv.fp32_precision = 'tf32'
except AttributeError:
    pass

# ----------------------------
# 配置：两套模型
# ----------------------------
MODEL_CATALOG = {
    "⚡ uint4 (更快/预览)": {
        "repo_id": "Disty0/Z-Image-Turbo-SDNQ-uint4-svd-r32",
        "default_local_path": "/content/Z-Image-Turbo-SDNQ-uint4",
    },
    "🎯 int8 (更好/产品)": {
        "repo_id": "Disty0/Z-Image-Turbo-SDNQ-int8",
        "default_local_path": "/content/Z-Image-Turbo-SDNQ-int8",
    },
}

# 必须下载的文件列表（两套模型一致）
REQUIRED_FILES = [
    "model_index.json",
    "scheduler/scheduler_config.json",
    "text_encoder/config.json",
    "text_encoder/generation_config.json",
    "text_encoder/model.safetensors",
    "text_encoder/quantization_config.json",
    "tokenizer/merges.txt",
    "tokenizer/tokenizer.json",
    "tokenizer/tokenizer_config.json",
    "tokenizer/vocab.json",
    "transformer/config.json",
    "transformer/diffusion_pytorch_model.safetensors",
    "transformer/quantization_config.json",
    "vae/config.json",
    "vae/diffusion_pytorch_model.safetensors",
]

# ----------------------------
# UI 组件
# ----------------------------
output = widgets.Output()

model_dropdown = widgets.Dropdown(
    options=list(MODEL_CATALOG.keys()),
    value="⚡ uint4 (更快/预览)",
    description="选择模型:",
    layout=widgets.Layout(width="420px"),
)

repo_html = widgets.HTML(value="")

path_input = widgets.Text(
    value=MODEL_CATALOG[model_dropdown.value]["default_local_path"],
    placeholder="本地模型保存/加载路径",
    description="模型路径:",
    layout=widgets.Layout(width="98%")
)

# ✅ 新增：可选外置 VAE 路径
vae_path_input = widgets.Text(
    value="",
    placeholder="（可选）留空=用模型自带VAE；不空=加载该路径VAE(可填模型根或vae子目录)",
    description="VAE路径:",
    layout=widgets.Layout(width="98%")
)
vae_status_html = widgets.HTML(value="<span style='color:gray;'>VAE：默认（使用模型自带）</span>")

mode_dropdown = widgets.Dropdown(
    options=[
        ("🌐 从 HuggingFace 在线加载", "online"),
        ("📁 从本地路径加载", "local"),
        ("⬇️ 下载模型到本地", "download"),
        ("🔧 检查并修复本地模型", "repair"),
    ],
    value="local",
    description="操作模式:",
    layout=widgets.Layout(width="350px")
)

execute_btn = widgets.Button(
    description="🚀 执行",
    button_style="primary",
    layout=widgets.Layout(width="120px", height="40px")
)

cleanup_btn = widgets.Button(
    description="🧹 一键清除旧模型",
    button_style="warning",
    layout=widgets.Layout(width="160px", height="40px")
)

status_html = widgets.HTML(value="<span style='color:gray;'>准备就绪</span>")
progress_html = widgets.HTML(value="")

local_status_html = widgets.HTML(value="")

# ----------------------------
# 小工具
# ----------------------------
def current_model_cfg():
    return MODEL_CATALOG[model_dropdown.value]

def update_repo_info():
    cfg = current_model_cfg()
    repo_html.value = f"<div style='font-size:12px;color:#666;'><b>Repo:</b> {cfg['repo_id']}</div>"

def check_local_model(path):
    if not os.path.exists(path):
        return False, "路径不存在", []
    missing_files = []
    for f in REQUIRED_FILES:
        fp = os.path.join(path, f)
        if not os.path.exists(fp) or os.path.getsize(fp) == 0:
            missing_files.append(f)
    if missing_files:
        return False, f"缺失 {len(missing_files)} 个文件", missing_files
    return True, "模型完整", []

def refresh_local_status():
    p = path_input.value.strip()
    is_valid, msg, missing = check_local_model(p)
    if is_valid:
        local_status_html.value = "<span style='color:green;'>✅ 本地模型完整，可直接加载</span>"
    elif os.path.exists(p):
        local_status_html.value = f"<span style='color:orange;'>⚠️ 本地模型不完整：{msg}</span>"
    else:
        local_status_html.value = "<span style='color:gray;'>📭 本地无模型，请先下载</span>"

def _resolve_vae_dir(user_path: str) -> str:
    """
    允许用户填：
    - 直接 vae 目录
    - 或模型根目录（里面有 vae/）
    返回真正的 vae 目录路径
    """
    p = (user_path or "").strip()
    if not p:
        return ""

    if not os.path.exists(p):
        raise RuntimeError(f"VAE 路径不存在: {p}")

    # 1) 直接就是 vae 目录
    if os.path.isdir(p) and os.path.exists(os.path.join(p, "config.json")):
        return p

    # 2) 是模型根目录，尝试 root/vae
    cand = os.path.join(p, "vae")
    if os.path.isdir(cand) and os.path.exists(os.path.join(cand, "config.json")):
        return cand

    # 3) 兜底：走一层扫描（防止用户给到奇怪结构）
    for dirpath, _, filenames in os.walk(p):
        if "config.json" in filenames:
            # 权重文件名按 diffusers 常见命名
            if ("diffusion_pytorch_model.safetensors" in filenames) or ("diffusion_pytorch_model.bin" in filenames):
                if os.path.basename(dirpath).lower() == "vae":
                    return dirpath

    raise RuntimeError(
        f"在该路径下找不到可用的 VAE 目录（需要 config.json + diffusion_pytorch_model.safetensors/.bin）: {p}"
    )

def refresh_vae_status():
    p = (vae_path_input.value or "").strip()
    if not p:
        vae_status_html.value = "<span style='color:gray;'>VAE：默认（使用模型自带）</span>"
        return
    try:
        vd = _resolve_vae_dir(p)
        vae_status_html.value = f"<span style='color:green;'>✅ VAE：将使用外置路径</span><br><span style='color:#666;font-size:12px;'>resolved: {vd}</span>"
    except Exception as e:
        vae_status_html.value = f"<span style='color:red;'>❌ VAE 路径无效：{e}</span>"

def on_model_change(change):
    cfg = current_model_cfg()
    path_input.value = cfg["default_local_path"]
    update_repo_info()
    refresh_local_status()

model_dropdown.observe(on_model_change, names="value")
update_repo_info()

vae_path_input.observe(lambda c: refresh_vae_status(), names="value")
refresh_vae_status()

# ----------------------------
# 下载/修复
# ----------------------------
def download_file(repo_id, filename, save_path):
    from huggingface_hub import hf_hub_download
    try:
        hf_hub_download(
            repo_id=repo_id,
            filename=filename,
            local_dir=save_path,
            local_dir_use_symlinks=False,
            force_download=True,
        )
        local_file = os.path.join(save_path, filename)
        if os.path.exists(local_file) and os.path.getsize(local_file) > 0:
            return True, os.path.getsize(local_file)
        return False, 0
    except Exception as e:
        return False, str(e)

def download_all_files(repo_id, save_path):
    os.makedirs(save_path, exist_ok=True)
    results = {"success": 0, "failed": 0, "skipped": 0}
    failed_files = []

    for i, filename in enumerate(REQUIRED_FILES):
        local_file = os.path.join(save_path, filename)
        progress_html.value = f"<span style='color:blue;'>📥 [{i+1}/{len(REQUIRED_FILES)}] {filename}</span>"

        if os.path.exists(local_file) and os.path.getsize(local_file) > 0:
            size_mb = os.path.getsize(local_file) / (1024 * 1024)
            print(f"⏭️ 已存在: {filename} ({size_mb:.2f} MB)")
            results["skipped"] += 1
            continue

        print(f"⬇️ 下载: {filename}...")
        success, info = download_file(repo_id, filename, save_path)
        if success:
            print(f"   ✅ 完成 ({info/(1024*1024):.2f} MB)")
            results["success"] += 1
        else:
            print(f"   ❌ 失败: {info}")
            results["failed"] += 1
            failed_files.append(filename)

    progress_html.value = ""
    return results, failed_files

def repair_model(repo_id, save_path):
    is_valid, msg, missing = check_local_model(save_path)
    if is_valid:
        print("✅ 模型已完整，无需修复")
        return True

    print(f"🔧 发现 {len(missing)} 个缺失文件，开始修复...")
    print("-" * 40)

    ok = 0
    for i, filename in enumerate(missing):
        progress_html.value = f"<span style='color:orange;'>🔧 [{i+1}/{len(missing)}] {filename}</span>"
        print(f"⬇️ 下载: {filename}...")
        success, info = download_file(repo_id, filename, save_path)
        if success:
            print(f"   ✅ 完成 ({info/(1024*1024):.2f} MB)")
            ok += 1
        else:
            print(f"   ❌ 失败: {info}")

    progress_html.value = ""
    print("-" * 40)
    print(f"📊 修复完成: {ok}/{len(missing)}")

    is_valid, _, _ = check_local_model(save_path)
    return is_valid

# ----------------------------
# 🧹 一键清除（释放旧pipe/缓存）
# ----------------------------
def hard_cleanup(drop_loaded_components: bool = True):
    global pipe
    freed = []

    if "pipe" in globals() and pipe is not None:
        try:
            del pipe
            freed.append("pipe")
        except Exception:
            pass
        finally:
            pipe = None

    if drop_loaded_components and "loaded_components" in globals():
        try:
            del globals()["loaded_components"]
            freed.append("loaded_components")
        except Exception:
            pass

    if torch.cuda.is_available():
        try:
            torch.cuda.empty_cache()
        except Exception:
            pass
        try:
            torch.cuda.ipc_collect()
        except Exception:
            pass

    try:
        gc.collect()
    except Exception:
        pass

    return freed

def on_cleanup_click(b):
    output.clear_output()
    cleanup_btn.disabled = True
    with output:
        try:
            status_html.value = "<span style='color:orange;'>🧹 清理中...</span>"
            freed = hard_cleanup(drop_loaded_components=True)
            print("🧹 清理完成")
            print("释放对象:", ", ".join(freed) if freed else "(无可释放对象 / 已是干净状态)")

            if torch.cuda.is_available():
                allocated = torch.cuda.memory_allocated() / (1024**3)
                reserved = torch.cuda.memory_reserved() / (1024**3)
                print(f"💾 显存: {allocated:.2f} GB (使用) / {reserved:.2f} GB (保留)")

            status_html.value = "<span style='color:green;'>✅ 已清理</span>"
        except Exception as e:
            print("❌ 清理失败:", e)
            status_html.value = "<span style='color:red;'>❌ 清理失败</span>"
        finally:
            cleanup_btn.disabled = False

cleanup_btn.on_click(on_cleanup_click)

# ----------------------------
# TE 按需加载/卸载（供“生latent单元”调用）
# ----------------------------
def set_pipe_model_root(pipe, model_path: str):
    pipe._model_root = model_path

def drop_text_encoder(pipe, drop_tokenizer: bool = False):
    try:
        te = getattr(pipe, "text_encoder", None)
        pipe.text_encoder = None
        if te is not None:
            del te
    except Exception:
        pass

    if drop_tokenizer:
        try:
            tok = getattr(pipe, "tokenizer", None)
            pipe.tokenizer = None
            if tok is not None:
                del tok
        except Exception:
            pass

    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def ensure_text_encoder_loaded(pipe, device: str = "cpu", dtype=torch.float32):
    if getattr(pipe, "text_encoder", None) is not None:
        return pipe.text_encoder

    root = getattr(pipe, "_model_root", None)
    if root is None:
        raise RuntimeError("pipe._model_root 未设置，无法按需加载 text_encoder")

    te_dir = os.path.join(root, "text_encoder")
    if not os.path.exists(te_dir):
        raise RuntimeError(
            f"找不到本地 text_encoder 目录: {te_dir}"
            f"提示：按需TE建议使用“本地加载模式”，或先把模型下载到本地。"
        )

    from transformers import AutoModel
    from sdnq.loader import apply_sdnq_options_to_model

    te = AutoModel.from_pretrained(te_dir, torch_dtype=dtype, local_files_only=True)
    te = apply_sdnq_options_to_model(te, use_quantized_matmul=True)

    try:
        te.to(device)
    except Exception:
        pass

    pipe.text_encoder = te
    return te

def ensure_tokenizer_loaded(pipe):
    if getattr(pipe, "tokenizer", None) is not None:
        return pipe.tokenizer

    root = getattr(pipe, "_model_root", None)
    if root is None:
        raise RuntimeError("pipe._model_root 未设置，无法按需加载 tokenizer")

    tok_dir = os.path.join(root, "tokenizer")
    if not os.path.exists(tok_dir):
        raise RuntimeError(
            f"找不到本地 tokenizer 目录: {tok_dir}"
            f"提示：按需 tokenizer 建议使用“本地加载模式”，或先把模型下载到本地。"
        )

    from transformers import AutoTokenizer
    tok = AutoTokenizer.from_pretrained(tok_dir, local_files_only=True)
    pipe.tokenizer = tok
    return tok

# ----------------------------
# ✅ 新增：外置 VAE 加载并覆盖 pipe.vae
# ----------------------------
def maybe_override_vae(pipe, vae_path_text: str, local_only_hint: bool):
    """
    vae_path_text 为空：不改动（使用模型自带 VAE）
    vae_path_text 不为空：从该路径加载 VAE 并覆盖 pipe.vae
    local_only_hint：当前加载模式是否 local_only（在线模式也允许外置VAE=本地）
    """
    vae_path_text = (vae_path_text or "").strip()
    if not vae_path_text:
        return False, "VAE: 默认（使用模型自带）"

    vae_dir = _resolve_vae_dir(vae_path_text)

    from diffusers import AutoencoderKL

    # 外置VAE一般是本地目录，所以这里强制 local_files_only=True 更稳
    # dtype：你当前 pipeline 用 FP32 基线；但 VAE 通常 FP16 解码更省显存
    # 这里加载为 float16，不影响 latent 兼容性（只是解码阶段）
    vae = AutoencoderKL.from_pretrained(
        vae_dir,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        local_files_only=True,
    )

    # 覆盖旧 VAE
    old = getattr(pipe, "vae", None)
    pipe.vae = vae
    try:
        if torch.cuda.is_available():
            pipe.vae.to("cuda")
    except Exception:
        pass

    try:
        if old is not None:
            del old
    except Exception:
        pass

    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

    return True, f"VAE: 已覆盖为外置路径 -> {vae_dir}"

# ----------------------------
# 🔥 cache-dit 初始化
# ----------------------------
# 替换原有的 try_enable_cache_dit 函数
def try_enable_cache_dit(pipe):
    try:
        import cache_dit
        from cache_dit import ForwardPattern, BlockAdapter, DBCacheConfig

        # T4 16GB保守配置
        cache_config = DBCacheConfig(
            Fn_compute_blocks=1,
            residual_diff_threshold=0.3,  # 更高阈值，更激进缓存
            max_warmup_steps=1,
            max_cached_steps=25,
        )

        cache_dit.enable_cache(
            BlockAdapter(pipe=pipe, auto=True, forward_pattern=ForwardPattern.Pattern_1),
            cache_config=cache_config
        )
        print("✅ cache-dit: T4-optimized config enabled")
        return True
    except Exception as e:
        print(f"⚠️ cache-dit failed: {e}")
        return False

# ----------------------------
# 加载模型：固定不常驻TE（保持其余加载逻辑/参数不变）
# ----------------------------
def load_model(model_id_or_path, local_only=False, keep_text_encoder=False):
    global pipe

    import diffusers
    from sdnq.loader import apply_sdnq_options_to_model

    load_kwargs = {
        "torch_dtype": torch.float32,
        "device_map": "cuda",
    }
    if local_only:
        load_kwargs["local_files_only"] = True

    pipe = diffusers.ZImagePipeline.from_pretrained(model_id_or_path, **load_kwargs)
    set_pipe_model_root(pipe, model_id_or_path)

    pipe.transformer = apply_sdnq_options_to_model(pipe.transformer, use_quantized_matmul=True)

    # ✅ 不常驻TE
    drop_text_encoder(pipe, drop_tokenizer=False)

    # 🔥 关键：启用 cache-dit
    try_enable_cache_dit(pipe)

    return pipe

# ----------------------------
# UI 回调
# ----------------------------
def on_execute_click(b):
    global pipe

    output.clear_output()
    execute_btn.disabled = True
    mode = mode_dropdown.value

    keep_te = False  # 固定不常驻TE

    cfg = current_model_cfg()
    repo_id = cfg["repo_id"]
    local_path = path_input.value.strip()
    vae_path = vae_path_input.value.strip()

    with output:
        try:
            if mode == "download":
                print(f"⬇️ 开始下载模型\n📦 模型: {repo_id}\n💾 保存到: {local_path}\n" + "=" * 50)
                status_html.value = "<span style='color:orange;'>⏳ 下载中...</span>"

                results, failed = download_all_files(repo_id, local_path)

                print("=" * 50)
                print(f"📊 结果: 下载 {results['success']}, 跳过 {results['skipped']}, 失败 {results['failed']}")
                if results["failed"] > 0:
                    print("\n❌ 失败的文件:")
                    for f in failed:
                        print(f"   - {f}")
                    status_html.value = f"<span style='color:orange;'>⚠️ {results['failed']} 个文件下载失败</span>"
                else:
                    total_size = sum(
                        os.path.getsize(os.path.join(local_path, f))
                        for f in REQUIRED_FILES if os.path.exists(os.path.join(local_path, f))
                    )
                    print(f"\n✅ 下载完成! 总大小: {total_size / (1024**3):.2f} GB")
                    status_html.value = f"<span style='color:green;'>✅ 下载完成 ({total_size / (1024**3):.2f} GB)</span>"

            elif mode == "repair":
                print(f"🔧 检查并修复模型\n📦 模型: {repo_id}\n📂 路径: {local_path}\n" + "=" * 50)
                status_html.value = "<span style='color:orange;'>⏳ 检查中...</span>"

                is_valid, msg, missing = check_local_model(local_path)
                print(f"当前状态: {msg}")
                if missing:
                    print("\n缺失的文件:")
                    for f in missing:
                        print(f"   ❌ {f}")
                    print()

                if repair_model(repo_id, local_path):
                    status_html.value = "<span style='color:green;'>✅ 模型已完整</span>"
                else:
                    status_html.value = "<span style='color:red;'>❌ 仍有文件缺失</span>"

            elif mode == "local":
                print(f"📁 从本地加载模型\n📦 模型: {repo_id}\n📂 路径: {local_path}\n" + "-" * 50)
                is_valid, msg, missing = check_local_model(local_path)
                print(f"模型状态: {msg}")
                if not is_valid:
                    print("\n❌ 模型不完整! 缺失文件:")
                    for f in missing:
                        print(f"   - {f}")
                    print("\n💡 先用「修复」或「下载」")
                    status_html.value = "<span style='color:red;'>❌ 模型不完整</span>"
                    return

                status_html.value = "<span style='color:orange;'>⏳ 加载中...</span>"

                if "pipe" in globals() and pipe is not None:
                    del pipe
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                    gc.collect()

                pipe = load_model(local_path, local_only=True, keep_text_encoder=keep_te)

                # ✅ 关键：根据输入框决定是否覆盖VAE
                changed, vae_msg = maybe_override_vae(pipe, vae_path, local_only_hint=True)
                print("✅ 模型加载完成!")
                print(f"🧠 TE 常驻: {keep_te}")
                print(f"📌 当前模型: {repo_id}")
                print(f"🧩 {vae_msg}")
                status_html.value = "<span style='color:green;'>✅ 模型已加载 (本地)</span>"

            elif mode == "online":
                print(f"🌐 从 HuggingFace 在线加载\n📦 模型: {repo_id}\n" + "-" * 50)
                status_html.value = "<span style='color:orange;'>⏳ 加载中（首次需下载）...</span>"

                if "pipe" in globals() and pipe is not None:
                    del pipe
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                    gc.collect()

                pipe = load_model(repo_id, local_only=False, keep_text_encoder=keep_te)

                # ✅ 在线也允许覆盖VAE（前提是你给的VAE路径在本地）
                changed, vae_msg = maybe_override_vae(pipe, vae_path, local_only_hint=False)

                print("✅ 模型加载完成!")
                print(f"🧠 TE 常驻: {keep_te}")
                print(f"📌 当前模型: {repo_id}")
                print(f"🧩 {vae_msg}")

                if not keep_te:
                    print("⚠️ 提示：在线加载 + TE按需加载 组合可能无法找到本地 text_encoder/tokenizer 目录。")
                    print("   建议：先下载到本地，再用“本地加载 + TE按需”来跑两阶段流程。")
                status_html.value = "<span style='color:green;'>✅ 模型已加载 (在线)</span>"

            # 显存显示
            if torch.cuda.is_available() and "pipe" in globals() and pipe is not None:
                allocated = torch.cuda.memory_allocated() / (1024**3)
                reserved = torch.cuda.memory_reserved() / (1024**3)
                print(f"\n💾 显存: {allocated:.2f} GB (使用) / {reserved:.2f} GB (保留)")

        except Exception as e:
            print(f"\n❌ 错误: {str(e)}")
            status_html.value = "<span style='color:red;'>❌ 失败</span>"
            import traceback
            traceback.print_exc()
        finally:
            execute_btn.disabled = False
            progress_html.value = ""
            refresh_vae_status()

execute_btn.on_click(on_execute_click)

# 本地状态联动
path_input.observe(lambda c: refresh_local_status(), names="value")
refresh_local_status()

# ----------------------------
# UI 布局
# ----------------------------
header = widgets.HTML(value="""
<div style='background:linear-gradient(135deg,#11998e 0%,#38ef7d 100%);
            color:white; padding:12px 15px; border-radius:8px; margin-bottom:10px;'>
    <h3 style='margin:0;'>🤖 模型管理器</h3>
    <span style='font-size:12px;'>Z-Image-Turbo-SDNQ | 双模型（uint4预览/int8产品）| TE按需加载 + 一键清除 + 外置VAE可选</span>
</div>
""")

layout = widgets.VBox([
    header,
    widgets.HBox([model_dropdown]),
    repo_html,
    local_status_html,
    widgets.HTML(value="<hr style='margin:10px 0;'>"),
    mode_dropdown,
    path_input,
    vae_path_input,
    vae_status_html,
    widgets.HBox([execute_btn, cleanup_btn, status_html]),
    progress_html,
    output
], layout=widgets.Layout(
    padding="10px",
    border="1px solid #ddd",
    border_radius="10px",
    width="760px"
))

display(layout)

In [ ]:
# @title 🎛️ Z-Image Turbo 出图面板
import os, gc, time, json, numpy as np, torch, traceback, inspect
from datetime import datetime
from PIL import Image
from IPython.display import display, clear_output
import ipywidgets as widgets
from google.colab import drive

# ========= 依赖：模型管理器的 3 个函数 =========
def _require_manager_funcs():
    needed = ["ensure_tokenizer_loaded", "ensure_text_encoder_loaded", "drop_text_encoder"]
    missing = [n for n in needed if n not in globals()]
    if missing:
        raise RuntimeError("缺少模型管理器函数（先运行模型管理器并加载模型）: " + ", ".join(missing))
    return (globals()["ensure_tokenizer_loaded"],
            globals()["ensure_text_encoder_loaded"],
            globals()["drop_text_encoder"])

def _get_existing_pipe():
    if "pipe" in globals() and globals().get("pipe", None) is not None:
        return globals()["pipe"]
    if "loaded_components" in globals() and isinstance(globals()["loaded_components"], dict):
        return globals()["loaded_components"].get("pipe", None)
    return None

def _round_to_16(x: int) -> int:
    return int(round(x / 16) * 16)

def _pipe_accepts_kw(p, key: str) -> bool:
    try:
        sig = inspect.signature(p.__call__)
        return key in sig.parameters
    except Exception:
        return False

# ========= allocator/显存小工具 =========
def _get_allocator_conf():
    return os.environ.get("PYTORCH_CUDA_ALLOC_CONF", "")

def _allocator_status_text():
    conf = _get_allocator_conf()
    if not conf:
        return ("<span style='color:orange;'>⚠️ allocator: 未设置 PYTORCH_CUDA_ALLOC_CONF</span>"
                "<br><span style='color:#64748b;font-size:12px;'>建议重启 runtime 后最先设置："
                "expandable_segments:True,max_split_size_mb:128</span>")
    ok = ("expandable_segments:True" in conf)
    if ok:
        return (f"<span style='color:green;'>✅ allocator: {conf}</span>"
                "<br><span style='color:#64748b;font-size:12px;'>已启用 expandable_segments（更抗碎片，适合冲大图）</span>")
    return (f"<span style='color:orange;'>⚠️ allocator: {conf}</span>"
            "<br><span style='color:#64748b;font-size:12px;'>建议包含 expandable_segments:True（需要重启后生效）</span>")

def _print_cuda_mem_line(tag=""):
    if not torch.cuda.is_available():
        print("CUDA 不可用")
        return
    alloc = torch.cuda.memory_allocated() / (1024**3)
    resv  = torch.cuda.memory_reserved() / (1024**3)
    peak  = torch.cuda.max_memory_allocated() / (1024**3)
    if tag:
        print(f"[{tag}] ", end="")
    print(f"allocated={alloc:.2f} GB | reserved={resv:.2f} GB | peak={peak:.2f} GB")

def runtime_fragmentation_cleanup(synchronize=True, reset_peak=False):
    """运行时碎片整理：对当前进程有效（即使 allocator 没生效也有收益）"""
    if not torch.cuda.is_available():
        return
    if synchronize:
        try: torch.cuda.synchronize()
        except Exception: pass
    try: gc.collect()
    except Exception: pass
    try: torch.cuda.empty_cache()
    except Exception: pass
    try: torch.cuda.ipc_collect()
    except Exception: pass
    if reset_peak:
        try: torch.cuda.reset_peak_memory_stats()
        except Exception: pass
    if synchronize:
        try: torch.cuda.synchronize()
        except Exception: pass

# ========= SDPA 上下文 =========
def _enter_sdpa_efficient():
    try:
        from torch.nn.attention import sdpa_kernel, SDPBackend
        ctx = sdpa_kernel([SDPBackend.EFFICIENT_ATTENTION, SDPBackend.MATH])
        ctx.__enter__()
        return ctx
    except Exception:
        return None

def _exit_sdpa(ctx):
    if ctx is None:
        return
    try:
        ctx.__exit__(None, None, None)
    except Exception:
        pass

# ========= dtype / 小工具（保留：不动基线） =========
def _dtype_from_mode(mode: str):
    mode = (mode or "fp32").lower()
    if mode == "fp16":
        return torch.float16
    if mode == "bf16":
        return torch.bfloat16
    return torch.float32

def _bf16_is_supported_cuda():
    if not torch.cuda.is_available():
        return False
    try:
        return bool(torch.cuda.is_bf16_supported())
    except Exception:
        return False

def _maybe_cast_pipe_modules(p, target_dtype: torch.dtype, enable: bool):
    if not enable:
        return
    if not torch.cuda.is_available():
        return
    try:
        if hasattr(p, "transformer") and p.transformer is not None:
            p.transformer.to("cuda", dtype=target_dtype)
    except Exception:
        pass
    try:
        if hasattr(p, "vae") and p.vae is not None:
            p.vae.to("cuda", dtype=target_dtype)
    except Exception:
        pass

# ========= 核心推理/解码逻辑 =========
def _prepare_full_gpu(p, attn_slice):
    if hasattr(p, "reset_device_map"):
        try: p.reset_device_map()
        except Exception: pass
    if not torch.cuda.is_available():
        raise RuntimeError("没有检测到 CUDA")

    runtime_fragmentation_cleanup(synchronize=True, reset_peak=False)

    p.to("cuda")
    if hasattr(p, "enable_attention_slicing"):
        if attn_slice == "auto":
            attn_slice = "max"
        p.enable_attention_slicing(attn_slice)

def _apply_int8_matmul_to_transformer(p, enable: bool):
    try:
        from sdnq.loader import apply_sdnq_options_to_model
    except Exception as e:
        return False, f"未能导入 sdnq.loader: {e}"
    if not hasattr(p, "transformer") or p.transformer is None:
        return False, "pipe.transformer 不存在"
    try:
        p.transformer = apply_sdnq_options_to_model(p.transformer, use_quantized_matmul=bool(enable))
        return True, f"INT8 MatMul = {bool(enable)}"
    except Exception as e:
        return False, f"设置失败: {e}"

# ========= CFG>1 时必须提供 negative_prompt_embeds =========
def _encode_prompt_embeds_then_drop_te(p, prompt_text: str, negative_text: str | None, cfg: float, te_dtype=torch.float32):
    ensure_tokenizer_loaded_fn, ensure_text_encoder_loaded_fn, drop_text_encoder_fn = _require_manager_funcs()

    if not _pipe_accepts_kw(p, "prompt_embeds"):
        raise RuntimeError("pipe 不支持 prompt_embeds")

    ensure_tokenizer_loaded_fn(p)

    te = ensure_text_encoder_loaded_fn(p, device="cuda", dtype=torch.float32)
    try:
        te.to("cuda", dtype=te_dtype)
    except Exception:
        pass

    encode_fn = getattr(p, "encode_prompt", None) or getattr(p, "_encode_prompt", None)
    if encode_fn is None:
        raise RuntimeError("pipe 没有 encode_prompt/_encode_prompt")

    sig = inspect.signature(encode_fn)
    keys = set(sig.parameters.keys())

    def _encode_one(text: str):
        kwargs = {}
        if "prompt" in keys:
            kwargs["prompt"] = text
        if "device" in keys:
            kwargs["device"] = torch.device("cuda")
        if "num_images_per_prompt" in keys:
            kwargs["num_images_per_prompt"] = 1
        if "do_classifier_free_guidance" in keys:
            kwargs["do_classifier_free_guidance"] = False
        if "negative_prompt" in keys:
            kwargs["negative_prompt"] = None

        encoded = encode_fn(**kwargs)

        out = {}
        if isinstance(encoded, torch.Tensor):
            out["prompt_embeds"] = encoded
        elif isinstance(encoded, (tuple, list)):
            out["prompt_embeds"] = encoded[0]
            if len(encoded) >= 3 and isinstance(encoded[2], torch.Tensor):
                out["pooled_prompt_embeds"] = encoded[2]
        elif isinstance(encoded, dict):
            for k in ["prompt_embeds", "pooled_prompt_embeds", "attention_mask"]:
                if k in encoded:
                    out[k] = encoded[k]
        else:
            raise RuntimeError(f"未知 encode 返回类型: {type(encoded)}")
        return out

    do_cfg = bool(cfg is not None and float(cfg) > 1.0)

    pos = _encode_one(prompt_text)
    embeds_kwargs = {"prompt_embeds": pos["prompt_embeds"]}
    if "pooled_prompt_embeds" in pos:
        embeds_kwargs["pooled_prompt_embeds"] = pos["pooled_prompt_embeds"]

    if do_cfg:
        neg_text = negative_text if (negative_text and negative_text.strip()) else ""
        neg = _encode_one(neg_text)
        embeds_kwargs["negative_prompt_embeds"] = neg["prompt_embeds"]
        if "pooled_prompt_embeds" in neg:
            embeds_kwargs["negative_pooled_prompt_embeds"] = neg["pooled_prompt_embeds"]

    drop_text_encoder_fn(p, drop_tokenizer=False)

    for k, v in list(embeds_kwargs.items()):
        if isinstance(v, torch.Tensor):
            embeds_kwargs[k] = v.to("cuda")
    return embeds_kwargs

def _decode_latent_with_pipe_vae(p, latent_t: torch.Tensor, scaling_factor: float, tiled: bool, tile_size):
    if not hasattr(p, "vae") or p.vae is None:
        raise RuntimeError("pipe.vae 不存在")
    vae = p.vae

    vae.to("cuda", dtype=torch.float16)

    lat = latent_t
    if lat.dim() == 3:
        lat = lat.unsqueeze(0)
    lat = lat.to("cuda", dtype=torch.float16)
    with torch.no_grad():
        if tiled and hasattr(vae, "enable_tiling"):
            vae.enable_tiling()
            try:
                vae.tile_sample_size = int(tile_size)
            except Exception:
                pass
        decoded = vae.decode(lat / float(scaling_factor)).sample
    img = (decoded[0] / 2 + 0.5).clamp(0, 1).detach().cpu().permute(1, 2, 0).float().numpy()
    return Image.fromarray((img * 255).astype(np.uint8))

# ========= 图像比例预设 =========
ASPECT_RATIOS = {
    "自定义": None,
    "1:1 方形 (1024×1024)": (1024, 1024),
    "4:3 横屏 (1152×864)": (1152, 864),
    "3:4 竖屏 (864×1152)": (864, 1152),
    "16:9 宽屏 (1344×768)": (1344, 768),
    "9:16 手机屏 (768×1344)": (768, 1344),
    "3:2 照片横 (1216×816)": (1216, 816),
    "2:3 照片竖 (816×1216)": (816, 1216),
    "21:9 超宽 (1536×656)": (1536, 656),
}

def _auto_tile_for_size(w, h):
    m = max(w, h)
    if m >= 1536: return 512
    if m >= 1024: return 768
    return 1024

# ========= Drive 挂载 =========
if not os.path.exists("/content/drive"):
    drive.mount("/content/drive")

# ========= 样式定义 =========
STYLE_CSS = """
<style>
.zimg-panel { font-family: 'Segoe UI', sans-serif; }
.zimg-section {
    background: #1e293b; border-radius: 12px; padding: 16px; margin: 8px 0;
    border: 1px solid #334155;
}
.zimg-section-title {
    color: #38bdf8; font-size: 14px; font-weight: 600;
    margin-bottom: 12px; padding-bottom: 8px;
    border-bottom: 1px solid #334155;
}
.zimg-header {
    background: linear-gradient(135deg, #0ea5e9 0%, #8b5cf6 50%, #ec4899 100%);
    border-radius: 16px; padding: 20px; margin-bottom: 16px;
    box-shadow: 0 4px 20px rgba(14, 165, 233, 0.3);
}
.zimg-header h1 { color: white; font-size: 24px; font-weight: 700; margin: 0 0 4px 0; }
.zimg-header p { color: rgba(255,255,255,0.85); font-size: 12px; margin: 0; }
.zimg-label { color: #94a3b8; font-size: 12px; margin-bottom: 4px; }
.zimg-info { color: #64748b; font-size: 11px; margin-top: 4px; }
</style>
"""

# ========= UI 组件 =========
header_html = widgets.HTML(STYLE_CSS + """
<div class="zimg-panel">
<div class="zimg-header">
    <h1>🎨 Z-Image Turbo</h1>
    <p>TE 编码后卸载 · VAE 常驻显存 · SDPA(Efficient) · TE卸载后碎片整理 · 元数据完整保存</p>
</div>
</div>
""")

allocator_html = widgets.HTML(value=f"""
<div style="margin:10px 0 0 0; padding:10px 12px; border:1px solid #334155; border-radius:12px; background:#0b1220;">
  <div style="font-size:13px; color:#e2e8f0; font-weight:600; margin-bottom:4px;">🧠 Boot/Allocator 状态</div>
  <div style="font-size:13px;">{_allocator_status_text()}</div>
</div>
""")

# ==================== 1. 提示词区域 ====================
prompt_box = widgets.Textarea(
    value="a beautiful cat sitting on a windowsill, soft sunlight, detailed fur, photorealistic",
    placeholder="输入正向提示词...",
    layout=widgets.Layout(width="100%", height="300px"),
)
neg_box = widgets.Textarea(
    value="",
    placeholder="输入负面提示词（CFG≤1无效）...",
    layout=widgets.Layout(width="100%", height="60px"),
)

# ==================== ✅ 新增：从 PNG 一键读取元数据并回填 ====================
w_png_meta_path = widgets.Text(
    value="",
    placeholder="填写要读取元数据的 PNG 路径（例如 /content/drive/MyDrive/Z-image/Outputs/img_xxx.png）",
    description="PNG 路径",
    style={'description_width': '90px'},
    layout=widgets.Layout(width="100%")
)

btn_load_meta = widgets.Button(
    description="📥 一键读取 PNG 元数据并回填参数",
    button_style="info",
    layout=widgets.Layout(width="320px", height="36px")
)

btn_copy_meta_json = widgets.Button(
    description="📋 复制元数据 JSON 到剪贴板",
    button_style="",
    layout=widgets.Layout(width="240px", height="36px")
)

meta_status = widgets.HTML(value="<div class='zimg-info'>在此填 PNG 路径后，可一键读取 parameters 元数据并回填面板参数。</div>")

def _read_metadata_from_png(png_path: str) -> dict | None:
    """从 PNG 文件读取元数据（parameters JSON），与本 cell 的保存格式匹配"""
    try:
        img = Image.open(png_path)
        if hasattr(img, "info") and "parameters" in img.info:
            return json.loads(img.info["parameters"])
    except Exception as e:
        print(f"读取 PNG 元数据失败: {e}")
    return None

def _safe_float(v, default=None):
    try:
        if v is None: return default
        return float(v)
    except Exception:
        return default

def _safe_int(v, default=None):
    try:
        if v is None: return default
        return int(float(v))
    except Exception:
        return default

def _set_ratio_preset_if_exact_match(width: int, height: int) -> str:
    for name, wh in ASPECT_RATIOS.items():
        if wh is None:
            continue
        if int(wh[0]) == int(width) and int(wh[1]) == int(height):
            return name
    return "自定义"

def _apply_metadata_to_ui(meta: dict):
    # prompt / negative
    if isinstance(meta.get("prompt", None), str):
        prompt_box.value = meta["prompt"]
    if isinstance(meta.get("negative_prompt", None), str):
        neg_box.value = meta["negative_prompt"]

    # steps / cfg / seed
    steps = _safe_int(meta.get("steps", None), default=None)
    if steps is not None:
        w_steps.value = int(max(w_steps.min, min(w_steps.max, steps)))

    cfg = _safe_float(meta.get("cfg_scale", None), default=None)
    if cfg is not None:
        w_cfg.value = float(max(w_cfg.min, min(w_cfg.max, cfg)))

    seed = _safe_int(meta.get("seed", None), default=None)
    if seed is not None:
        w_seed.value = int(seed)

    # width / height
    width = _safe_int(meta.get("width", None), default=None)
    height = _safe_int(meta.get("height", None), default=None)
    if width is not None and height is not None:
        width16 = _round_to_16(int(width))
        height16 = _round_to_16(int(height))
        width16 = int(max(w_width.min, min(w_width.max, width16)))
        height16 = int(max(w_height.min, min(w_height.max, height16)))

        # 尽量匹配预设，否则自定义
        ratio_name = meta.get("aspect_ratio_preset", None)
        if not isinstance(ratio_name, str) or ratio_name not in ASPECT_RATIOS:
            ratio_name = _set_ratio_preset_if_exact_match(width16, height16)
        w_ratio.value = ratio_name

        # 如果是自定义，直接写入；如果是预设，比例联动会自动写入近似值，这里再强制对齐到元数据值
        w_width.value = width16
        w_height.value = height16
        _update_size_info(None)

    # SVH
    svh = meta.get("seed_variance_enhancer", {})
    if isinstance(svh, dict):
        enabled = bool(svh.get("enabled", False))
        w_svh_enable.value = enabled

        rp = _safe_float(svh.get("randomize_percent", None), default=None)
        if rp is not None:
            w_svh_randomize_percent.value = float(max(w_svh_randomize_percent.min, min(w_svh_randomize_percent.max, rp)))

        st = _safe_float(svh.get("strength", None), default=None)
        if st is not None:
            w_svh_strength.value = float(st)

        ni = svh.get("noise_insert", None)
        if isinstance(ni, str) and ni in [o for o in w_svh_noise_insert.options]:
            w_svh_noise_insert.value = ni

        sw = _safe_float(svh.get("steps_switchover_percent", None), default=None)
        if sw is not None:
            w_svh_steps_switchover_percent.value = float(max(w_svh_steps_switchover_percent.min, min(w_svh_steps_switchover_percent.max, sw)))

        ms = svh.get("mask_starts_at", None)
        if isinstance(ms, str) and ms in [o for o in w_svh_mask_starts_at.options]:
            w_svh_mask_starts_at.value = ms

        mp = _safe_float(svh.get("mask_percent", None), default=None)
        if mp is not None:
            w_svh_mask_percent.value = float(max(w_svh_mask_percent.min, min(w_svh_mask_percent.max, mp)))

        sm = svh.get("seed_mode", None)
        if isinstance(sm, str) and sm in [v for _, v in w_svh_seed_mode.options]:
            w_svh_seed_mode.value = sm

        sseed = _safe_int(svh.get("seed", None), default=None)
        if sseed is not None:
            w_svh_seed.value = int(sseed)

    # LoRA 槽位
    lora = meta.get("lora_slots", {})
    if isinstance(lora, dict):
        # 若元数据带了 folder，则回填
        lf = lora.get("lora_folder", None)
        if isinstance(lf, str) and lf.strip():
            w_lora_dir.value = lf.strip()
            _refresh_lora_dropdowns()

        def _set_if_in_options(dd: widgets.Dropdown, val: str):
            vals = [v for _, v in dd.options]
            dd.value = val if (val in vals) else ""

        f1 = lora.get("slot1_file", "") or ""
        f2 = lora.get("slot2_file", "") or ""
        f3 = lora.get("slot3_file", "") or ""

        _set_if_in_options(w_lora1, f1)
        _set_if_in_options(w_lora2, f2)
        _set_if_in_options(w_lora3, f3)

        w1 = _safe_float(lora.get("slot1_weight", None), default=None)
        w2 = _safe_float(lora.get("slot2_weight", None), default=None)
        w3 = _safe_float(lora.get("slot3_weight", None), default=None)
        if w1 is not None: w_lora1_w.value = float(max(w_lora1_w.min, min(w_lora1_w.max, w1)))
        if w2 is not None: w_lora2_w.value = float(max(w_lora2_w.min, min(w_lora2_w.max, w2)))
        if w3 is not None: w_lora3_w.value = float(max(w_lora3_w.min, min(w_lora3_w.max, w3)))

def _copy_text_to_clipboard_js(text: str):
    # Colab/Jupyter 通用：用 JS 写剪贴板（需要用户手势触发按钮点击，正好满足）
    from IPython.display import Javascript, display as _disp
    safe = json.dumps(str(text))
    _disp(Javascript(f"""
    (async () => {{
      try {{
        await navigator.clipboard.writeText({safe});
        console.log("copied");
      }} catch(e) {{
        console.error(e);
      }}
    }})()
    """))

def _on_load_meta(_):
    out.clear_output()
    with out:
        png_path = (w_png_meta_path.value or "").strip()
        if not png_path:
            print("❌ 请先填写 PNG 路径")
            return
        if not os.path.exists(png_path):
            print(f"❌ PNG 不存在: {png_path}")
            return
        meta = _read_metadata_from_png(png_path)
        if not meta:
            print("❌ 未读到 parameters 元数据（该 PNG 可能不是本面板保存的，或没有 parameters 字段）")
            return

        _apply_metadata_to_ui(meta)

        # 同时把完整 JSON 放入全局，方便复制/检查
        globals()["_last_loaded_png_metadata"] = meta

        # 更新提示
        meta_status.value = f"<div class='zimg-info'>✅ 已从 PNG 读取并回填：{os.path.basename(png_path)}</div>"
        print("✅ 元数据已回填到面板参数。")
        print("— 可直接点 🚀 生成图像 复现。")
        print("— 如需复制原始 JSON，点「复制元数据 JSON 到剪贴板」。")

def _on_copy_meta_json(_):
    meta = globals().get("_last_loaded_png_metadata", None)
    if not isinstance(meta, dict):
        meta_status.value = "<div class='zimg-info'>⚠️ 尚未读取任何 PNG 元数据；请先点击「一键读取」</div>"
        return
    text = json.dumps(meta, ensure_ascii=False, indent=2)
    _copy_text_to_clipboard_js(text)
    meta_status.value = "<div class='zimg-info'>📋 已将元数据 JSON 复制到剪贴板</div>"

btn_load_meta.on_click(_on_load_meta)
btn_copy_meta_json.on_click(_on_copy_meta_json)

# ==================== 2. 图像尺寸区域 ====================
_current_ratio = None
_updating = False

w_ratio = widgets.Dropdown(
    options=list(ASPECT_RATIOS.keys()),
    value="自定义",
    description="",
    layout=widgets.Layout(width="100%"),
)

w_width = widgets.IntSlider(
    value=1024, min=512, max=2560, step=16,
    description="宽度",
    style={'description_width': '60px'},
    layout=widgets.Layout(width="100%"),
    readout=True,
    continuous_update=False,
)
w_height = widgets.IntSlider(
    value=1024, min=512, max=2560, step=16,
    description="高度",
    style={'description_width': '60px'},
    layout=widgets.Layout(width="100%"),
    readout=True,
    continuous_update=False,
)
w_size_info = widgets.HTML('<div class="zimg-info">📐 最终尺寸: 1024 × 1024 (已对齐16像素)</div>')

def _parse_ratio_from_name(name):
    if name == "自定义":
        return None
    w, h = ASPECT_RATIOS[name]
    return w / h

def _set_initial_size_by_ratio(ratio):
    if ratio is None:
        return 1024, 1024
    if ratio >= 1:
        w = 1024
        h = round(w / ratio)
    else:
        h = 1024
        w = round(h * ratio)
    w = max(512, min(2560, _round_to_16(w)))
    h = max(512, min(2560, _round_to_16(h)))
    return w, h

def _on_ratio_change(change):
    global _current_ratio, _updating
    ratio_name = change["new"]
    ratio = _parse_ratio_from_name(ratio_name)
    _current_ratio = ratio

    if ratio is not None:
        w, h = _set_initial_size_by_ratio(ratio)
        _updating = True
        w_width.value = w
        w_height.value = h
        _updating = False
        _update_size_info(None)
    else:
        _update_size_info(None)

def _on_width_change(change):
    global _current_ratio, _updating
    if _updating or _current_ratio is None:
        _update_size_info(change)
        return
    new_w = _round_to_16(change["new"])
    new_h = _round_to_16(new_w / _current_ratio)
    new_h = max(512, min(2560, new_h))
    _updating = True
    w_height.value = new_h
    _updating = False
    _update_size_info(None)

def _on_height_change(change):
    global _current_ratio, _updating
    if _updating or _current_ratio is None:
        _update_size_info(change)
        return
    new_h = _round_to_16(change["new"])
    new_w = _round_to_16(new_h * _current_ratio)
    new_w = max(512, min(2560, new_w))
    _updating = True
    w_width.value = new_w
    _updating = False
    _update_size_info(None)

def _update_size_info(change):
    w16 = _round_to_16(w_width.value)
    h16 = _round_to_16(w_height.value)
    w_size_info.value = f'<div class="zimg-info">📐 最终尺寸: {w16} × {h16} (已对齐16像素)</div>'

w_ratio.observe(_on_ratio_change, names="value")
w_width.observe(_on_width_change, names="value")
w_height.observe(_on_height_change, names="value")

# ==================== 3. 生成参数区域 ====================
w_steps = widgets.IntSlider(
    value=9, min=5, max=20, step=1,
    description="采样步数",
    style={'description_width': '80px'},
    layout=widgets.Layout(width="100%"),
)
w_cfg = widgets.FloatSlider(
    value=0.0, min=0.0, max=2.0, step=0.01,
    description="CFG 强度",
    style={'description_width': '80px'},
    layout=widgets.Layout(width="100%"),
)
w_seed = widgets.IntText(
    value=-1,
    description="随机种子",
    style={'description_width': '80px'},
    layout=widgets.Layout(width="100%"),
)

# ==================== ✅ 3.5 批量张数（新增） ====================
w_batch = widgets.BoundedIntText(
    value=1, min=1, max=200, step=1,
    description="每次张数",
    style={'description_width': '80px'},
    layout=widgets.Layout(width="220px")
)

# ==================== 4. 显存优化区域 ====================
w_attn = widgets.Dropdown(
    options=["auto", "max", 1, 2, 4, 8, 16, 32],
    value="auto",
    description="注意力切片",
    style={'description_width': '100px'},
    layout=widgets.Layout(width="100%"),
)
w_int8 = widgets.Checkbox(
    value=True,
    description="启用 INT8 量化加速",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="100%"),
)

w_dtype_mode = widgets.Dropdown(
    options=[
        ("默认 FP32（基线）", "fp32"),
        ("省显存 FP16（推荐）", "fp16"),
        ("省显存 BF16（需GPU支持）", "bf16"),
    ],
    value="fp32",
    description="dtype",
    style={'description_width': '60px'},
    layout=widgets.Layout(width="100%")
)

w_dtype_apply = widgets.Checkbox(
    value=False,
    description="启用 dtype 下沉（transformer/TE 等尽力转为 FP16/BF16）",
    indent=False,
    layout=widgets.Layout(width="100%")
)

dtype_hint = widgets.HTML(
    value="<div class='zimg-info'>💡 默认关闭以不破坏 2048² 基线。</div>"
)

# ==================== 5. VAE 解码区域 ====================
w_scaling = widgets.FloatText(
    value=0.3611,
    description="缩放因子",
    style={'description_width': '80px'},
    layout=widgets.Layout(width="100%")
)
w_tiled = widgets.Checkbox(
    value=False,
    description="启用 VAE 分块解码（大图推荐）",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="100%")
)
w_tile = widgets.Dropdown(
    options=["auto", 64, 128, 256, 384, 512, 768, 1024],
    value="auto",
    description="分块大小",
    style={'description_width': '80px'},
    layout=widgets.Layout(width="100%")
)

# ==================== 6. 保存选项区域 ====================
w_save_latent = widgets.Checkbox(
    value=True,
    description="保存 Latent (.npz)",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="50%")
)
w_save_png = widgets.Checkbox(
    value=True,
    description="保存 PNG 图像",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="50%")
)
w_latent_dir = widgets.Text(
    value="/content/drive/MyDrive/Z-image/Latent",
    description="Latent 目录",
    style={'description_width': '90px'},
    layout=widgets.Layout(width="100%")
)
w_png_dir = widgets.Text(
    value="/content/drive/MyDrive/Z-image/Outputs",
    description="PNG 目录",
    style={'description_width': '90px'},
    layout=widgets.Layout(width="100%")
)

# ==================== ✅ 6.5. LoRA 三槽位（推理时应用，不改其它逻辑） ====================
w_lora_dir = widgets.Text(
    value=globals().get("LORA_FOLDER", "/content/drive/MyDrive/model"),
    description="LoRA 目录",
    style={'description_width': '90px'},
    layout=widgets.Layout(width="100%")
)

def _list_lora_files_for_ui(folder: str):
    folder = (folder or "").strip()
    if not folder or not os.path.exists(folder):
        return []
    files = []
    for fn in os.listdir(folder):
        if fn.endswith(".safetensors") or fn.endswith(".pt") or fn.endswith(".bin"):
            files.append(fn)
    return sorted(files)

def _adapter_name_for_slot(i: int) -> str:
    return f"lora_{i}"

def _pipe_has_adapter(p, adapter_name: str) -> bool:
    try:
        target = getattr(p, "unet", None) or getattr(p, "transformer", None)
        if target is not None and hasattr(target, "peft_config") and target.peft_config:
            return adapter_name in target.peft_config
    except Exception:
        pass
    return False

def _ensure_lora_loaded_if_needed(p, adapter_name: str, folder: str, file_name: str):
    if not file_name:
        return
    if _pipe_has_adapter(p, adapter_name):
        return
    folder = (folder or "").strip()
    fp = os.path.join(folder, file_name)
    if not os.path.exists(fp):
        raise RuntimeError(f"LoRA 文件不存在: {fp}")
    p.load_lora_weights(fp, adapter_name=adapter_name)

def _delete_lora_if_present(p, adapter_name: str):
    try:
        p.delete_adapters([adapter_name])
    except Exception:
        pass

def _apply_lora_slots_to_pipe(p, slot_files: list, slot_weights: list, lora_folder: str):
    for i in range(1, 4):
        fn = slot_files[i-1]
        adapter = _adapter_name_for_slot(i)
        if not fn:
            _delete_lora_if_present(p, adapter)

    for i in range(1, 4):
        fn = slot_files[i-1]
        adapter = _adapter_name_for_slot(i)
        if fn:
            _ensure_lora_loaded_if_needed(p, adapter, folder=lora_folder, file_name=fn)

    active_adapters = []
    active_weights = []
    for i in range(1, 4):
        fn = slot_files[i-1]
        w = float(slot_weights[i-1])
        if fn and w != 0.0:
            active_adapters.append(_adapter_name_for_slot(i))
            active_weights.append(w)

    if not active_adapters:
        try:
            if hasattr(p, "set_adapters"):
                p.set_adapters([], [])
        except Exception:
            pass
        return

    try:
        if hasattr(p, "set_adapters"):
            p.set_adapters(active_adapters, active_weights)
    except Exception:
        pass

def _make_lora_slot_ui(slot_idx: int):
    dd = widgets.Dropdown(
        options=[("（不加载）", "")],
        value="",
        description=f"槽位{slot_idx}",
        layout=widgets.Layout(width="100%")
    )
    wt = widgets.FloatSlider(
        value=0.0, min=-2.0, max=2.0, step=0.05,
        description="强度",
        style={'description_width': '60px'},
        layout=widgets.Layout(width="100%"),
        readout=True,
        continuous_update=False,
    )
    return dd, wt

w_lora1, w_lora1_w = _make_lora_slot_ui(1)
w_lora2, w_lora2_w = _make_lora_slot_ui(2)
w_lora3, w_lora3_w = _make_lora_slot_ui(3)

btn_lora_refresh = widgets.Button(
    description="🔄 刷新 LoRA 文件列表",
    button_style="info",
    layout=widgets.Layout(width="220px", height="32px")
)

def _refresh_lora_dropdowns(_=None):
    folder = w_lora_dir.value.strip()
    files = _list_lora_files_for_ui(folder)
    opts = [("（不加载）", "")] + [(fn, fn) for fn in files]

    cur1, cur2, cur3 = w_lora1.value, w_lora2.value, w_lora3.value
    w_lora1.options = opts
    w_lora2.options = opts
    w_lora3.options = opts

    values = [v for _, v in opts]
    w_lora1.value = cur1 if cur1 in values else ""
    w_lora2.value = cur2 if cur2 in values else ""
    w_lora3.value = cur3 if cur3 in values else ""

    try:
        ll = globals().get("loaded_loras", {})
        if isinstance(ll, dict):
            if ll.get("lora_1") in values: w_lora1.value = ll["lora_1"]
            if ll.get("lora_2") in values: w_lora2.value = ll["lora_2"]
            if ll.get("lora_3") in values: w_lora3.value = ll["lora_3"]
    except Exception:
        pass

btn_lora_refresh.on_click(_refresh_lora_dropdowns)

def _on_lora_dir_change(change):
    _refresh_lora_dropdowns()

w_lora_dir.observe(_on_lora_dir_change, names="value")
_refresh_lora_dropdowns()

lora_slots_ui = widgets.VBox([
    widgets.HTML('<div class="zimg-label">LoRA 槽位（推理前自动加载/卸载；槽位显示文件名，避免误用）</div>'),
    w_lora_dir,
    widgets.HTML("<div class='zimg-info'>⚠️ 注意：加载 LoRA 可能导致 2048² 显存不够；可降分辨率继续用。</div>"),
    btn_lora_refresh,
    widgets.VBox([w_lora1, w_lora1_w, w_lora2, w_lora2_w, w_lora3, w_lora3_w])
])

# ==================== ✅ 6.6 SeedVarianceEnhancer 折叠模块 ====================
w_svh_enable = widgets.Checkbox(
    value=False,
    description="启用 SeedVarianceEnhancer（对 prompt embeds 加噪增强多样性）",
    indent=False,
    layout=widgets.Layout(width="100%")
)

w_svh_randomize_percent = widgets.FloatSlider(
    value=50.0, min=1.0, max=100.0, step=1.0,
    description="randomize%",
    style={'description_width': '110px'},
    layout=widgets.Layout(width="100%"),
    readout=True,
    continuous_update=False
)

w_svh_strength = widgets.FloatText(
    value=20.0,
    description="strength",
    style={'description_width': '110px'},
    layout=widgets.Layout(width="100%")
)

w_svh_noise_insert = widgets.Dropdown(
    options=["noise on beginning steps", "noise on ending steps", "noise on all steps", "disabled"],
    value="noise on all steps",
    description="noise_insert",
    style={'description_width': '110px'},
    layout=widgets.Layout(width="100%")
)

w_svh_steps_switchover_percent = widgets.FloatSlider(
    value=20.0, min=1.0, max=99.0, step=1.0,
    description="switch%",
    style={'description_width': '110px'},
    layout=widgets.Layout(width="100%"),
    readout=True,
    continuous_update=False
)

w_svh_seed_mode = widgets.Dropdown(
    options=[("跟随出图种子", "follow"), ("自定义 SVH 种子", "custom")],
    value="follow",
    description="SVH seed",
    style={'description_width': '110px'},
    layout=widgets.Layout(width="100%")
)

w_svh_seed = widgets.IntText(
    value=0,
    description="seed",
    style={'description_width': '110px'},
    layout=widgets.Layout(width="100%")
)

w_svh_mask_starts_at = widgets.Dropdown(
    options=["beginning", "end"],
    value="beginning",
    description="mask_from",
    style={'description_width': '110px'},
    layout=widgets.Layout(width="100%")
)

w_svh_mask_percent = widgets.FloatSlider(
    value=0.0, min=0.0, max=99.0, step=1.0,
    description="mask%",
    style={'description_width': '110px'},
    layout=widgets.Layout(width="100%"),
    readout=True,
    continuous_update=False
)

w_svh_log_to_console = widgets.Checkbox(
    value=False,
    description="log_to_console（打印 embedding 统计等）",
    indent=False,
    layout=widgets.Layout(width="100%")
)

def _svh_controls_enabled(enable: bool):
    for w in [
        w_svh_randomize_percent, w_svh_strength, w_svh_noise_insert,
        w_svh_steps_switchover_percent, w_svh_seed_mode, w_svh_seed,
        w_svh_mask_starts_at, w_svh_mask_percent, w_svh_log_to_console
    ]:
        w.disabled = not bool(enable)

def _on_svh_enable_change(change):
    _svh_controls_enabled(change["new"])
    _on_svh_seed_mode_change({"new": w_svh_seed_mode.value})

def _on_svh_seed_mode_change(change):
    w_svh_seed.disabled = (change["new"] != "custom") or (not bool(w_svh_enable.value))

w_svh_enable.observe(_on_svh_enable_change, names="value")
w_svh_seed_mode.observe(_on_svh_seed_mode_change, names="value")
_svh_controls_enabled(w_svh_enable.value)
_on_svh_seed_mode_change({"new": w_svh_seed_mode.value})

def _svh_apply_to_embeds(
    embeds_kwargs: dict,
    *,
    enable: bool,
    randomize_percent: float,
    strength: float,
    noise_insert: str,
    steps_switchover_percent: float,
    seed: int,
    mask_starts_at: str,
    mask_percent: float,
    log_to_console: bool,
):
    if not enable:
        return embeds_kwargs
    if noise_insert == "disabled":
        return embeds_kwargs
    if strength == 0:
        return embeds_kwargs
    if "prompt_embeds" not in embeds_kwargs or not isinstance(embeds_kwargs["prompt_embeds"], torch.Tensor):
        return embeds_kwargs

    rp = max(1.0, min(100.0, float(randomize_percent))) / 100.0
    mp = max(0.0, min(99.0, float(mask_percent))) / 100.0

    if noise_insert != "noise on all steps":
        if log_to_console:
            print(f"⚠️ SVH: 当前推理管线不支持按步段切换（{noise_insert}），已按 'noise on all steps' 处理。")

    def _first_null_last_nonnull(seq_t: torch.Tensor):
        first_null = -1
        last_nonnull = -1
        null_seq = [0] * seq_t.size(1)
        for i in range(seq_t.size(1)):
            s = seq_t[:, i, :]
            is_all_zero = bool(torch.all(s == 0).item())
            null_seq[i] = 0 if is_all_zero else 1
            if not is_all_zero:
                last_nonnull = i
            if is_all_zero and first_null == -1:
                first_null = i
        return first_null, last_nonnull, null_seq

    def _apply_one(t: torch.Tensor, local_seed: int):
        device = t.device
        torch.manual_seed(int(local_seed))
        noise = torch.rand_like(t) * 2 * float(strength) - float(strength)

        torch.manual_seed(int(local_seed) + 1)
        noise_mask = torch.bernoulli(torch.ones_like(t) * rp).bool()

        first_null, last_nonnull, null_seq = _first_null_last_nonnull(t)

        if mp > 0 or (last_nonnull < t.size(1) - 1 and last_nonnull >= 0):
            if last_nonnull < t.size(1) - 1 and last_nonnull >= 0:
                seq_len = last_nonnull + 1
            else:
                seq_len = t.size(1)

            if mask_starts_at == "end":
                mask_start = seq_len - int(seq_len * mp)
                mask_end = t.size(1)
            else:
                mask_start = 0
                mask_end = int(seq_len * mp)

            prompt_mask = torch.arange(t.size(1), device=device).view(1, -1, 1).expand(t.size(0), -1, t.size(2))
            prompt_mask = (prompt_mask >= mask_start) & (prompt_mask < mask_end)

            if first_null > -1:
                null_mask_tensor = ~torch.tensor(null_seq, device=device, dtype=torch.bool)
                null_mask_tensor = null_mask_tensor.view(1, -1, 1).expand(t.size(0), -1, t.size(2))
                prompt_mask = prompt_mask | null_mask_tensor

            noise_mask = noise_mask & (~prompt_mask)

        out = t + (noise * noise_mask)

        if log_to_console:
            with torch.no_grad():
                st = float(torch.std(t).item())
                print(f"SVH: applied noise | seed={local_seed} | rp={rp:.2f} | strength={strength} | std={st:.6f} | suggest {st/10:.6f}-{st*10:.6f}")

        return out

    embeds_kwargs = dict(embeds_kwargs)
    embeds_kwargs["prompt_embeds"] = _apply_one(embeds_kwargs["prompt_embeds"], seed)

    if "negative_prompt_embeds" in embeds_kwargs and isinstance(embeds_kwargs["negative_prompt_embeds"], torch.Tensor):
        embeds_kwargs["negative_prompt_embeds"] = _apply_one(embeds_kwargs["negative_prompt_embeds"], seed + 999)

    return embeds_kwargs

# ==================== 7. 操作按钮 ====================
btn_run = widgets.Button(
    description="🚀 生成图像",
    button_style="success",
    layout=widgets.Layout(width="160px", height="48px"),
)
btn_clear = widgets.Button(
    description="🗑️ 清空输出",
    button_style="warning",
    layout=widgets.Layout(width="140px", height="48px"),
)

out = widgets.Output(layout=widgets.Layout(
    border="2px solid #334155",
    border_radius="12px",
    padding="16px",
    min_height="300px",
    background="#0f172a",
))

# ======================================================
# ✅ 元数据构建与写入（已修复）
# ======================================================
def _build_generation_metadata(
    prompt: str,
    negative: str,
    seed: int,
    steps: int,
    cfg: float,
    width: int,
    height: int,
    ratio_name: str,
    svh_meta: dict | None,
    lora_slots: dict | None,
    pipe=None,
) -> dict:
    """构建仅包含出图参数和 SVH 参数的元数据字典"""
    meta = {
        "prompt": prompt,
        "negative_prompt": negative,
        "seed": int(seed),
        "steps": int(steps),
        "cfg_scale": float(cfg),
        "width": int(width),
        "height": int(height),
        "aspect_ratio_preset": ratio_name,
        "lora_slots": lora_slots or {},
        "seed_variance_enhancer": svh_meta or {},
        "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    }

    if pipe is not None:
        try:
            meta["pipeline_class"] = pipe.__class__.__name__
        except Exception:
            pass
        try:
            if hasattr(pipe, "scheduler") and pipe.scheduler is not None:
                meta["scheduler_class"] = pipe.scheduler.__class__.__name__
        except Exception:
            pass
        loras = []
        try:
            target = getattr(pipe, "unet", None) or getattr(pipe, "transformer", None)
            if target is not None and hasattr(target, "peft_config") and target.peft_config:
                loras = list(target.peft_config.keys())
        except Exception:
            pass
        meta["active_lora_adapters"] = loras

    return meta

def _save_latent_with_metadata(
    latent: torch.Tensor,
    metadata: dict,
    save_path: str,
) -> bool:
    try:
        lat_np = latent.detach().cpu().to(torch.float32).numpy()
        metadata_json = json.dumps(metadata, ensure_ascii=False, indent=2)
        np.savez_compressed(
            save_path,
            latent=lat_np,
            metadata_json=np.array([metadata_json], dtype=object),
        )
        return True
    except Exception as e:
        print(f"❌ 保存 Latent 失败: {e}")
        traceback.print_exc()
        return False

def _save_png_with_metadata(
    image: Image.Image,
    metadata: dict,
    save_path: str,
) -> bool:
    try:
        from PIL import PngImagePlugin
        pnginfo = PngImagePlugin.PngInfo()

        metadata_json = json.dumps(metadata, ensure_ascii=False, indent=2)
        pnginfo.add_text("parameters", metadata_json)

        pnginfo.add_text("prompt", str(metadata.get("prompt", "")))
        pnginfo.add_text("negative_prompt", str(metadata.get("negative_prompt", "")))
        pnginfo.add_text("seed", str(metadata.get("seed", "")))
        pnginfo.add_text("steps", str(metadata.get("steps", "")))
        pnginfo.add_text("cfg_scale", str(metadata.get("cfg_scale", "")))
        pnginfo.add_text("width", str(metadata.get("width", "")))
        pnginfo.add_text("height", str(metadata.get("height", "")))

        svh = metadata.get("seed_variance_enhancer", {})
        if svh.get("enabled"):
            pnginfo.add_text("svh_enabled", "true")
            pnginfo.add_text("svh_strength", str(svh.get("strength", "")))
            pnginfo.add_text("svh_randomize_percent", str(svh.get("randomize_percent", "")))
            pnginfo.add_text("svh_seed", str(svh.get("seed", "")))

        lora_slots = metadata.get("lora_slots", {})
        for slot_key in ["slot1_file", "slot2_file", "slot3_file"]:
            if lora_slots.get(slot_key):
                pnginfo.add_text(f"lora_{slot_key}", str(lora_slots[slot_key]))

        image.save(save_path, pnginfo=pnginfo)
        return True
    except Exception as e:
        print(f"❌ 保存 PNG 失败: {e}")
        traceback.print_exc()
        return False

def _read_metadata_from_latent(npz_path: str) -> dict | None:
    try:
        data = np.load(npz_path, allow_pickle=True)
        if "metadata_json" in data:
            json_str = data["metadata_json"]
            if isinstance(json_str, np.ndarray):
                json_str = json_str.item() if json_str.ndim == 0 else json_str[0]
            return json.loads(json_str)
    except Exception as e:
        print(f"读取 latent 元数据失败: {e}")
    return None

# ========= 生成逻辑 =========
def _on_run(_):
    out.clear_output()
    with out:
        sdpa_ctx = None
        try:
            p = _get_existing_pipe()
            if p is None:
                print("❌ 未检测到 pipe：请先用模型管理器加载模型")
                return

            prompt = prompt_box.value.strip()
            negative = neg_box.value.strip()
            width = int(w_width.value)
            height = int(w_height.value)
            steps = int(w_steps.value)
            cfg = float(w_cfg.value)
            seed_in = int(w_seed.value)
            batch_n = int(w_batch.value)

            attn_slice = w_attn.value
            int8_enable = bool(w_int8.value)
            scaling = float(w_scaling.value)
            tiled = bool(w_tiled.value)
            tile_size = w_tile.value

            save_latent = bool(w_save_latent.value)
            save_png = bool(w_save_png.value)
            latent_dir = w_latent_dir.value.strip()
            png_dir = w_png_dir.value.strip()

            os.makedirs(latent_dir, exist_ok=True)
            os.makedirs(png_dir, exist_ok=True)

            width16 = _round_to_16(width)
            height16 = _round_to_16(height)

            if tile_size == "auto":
                tile_size = _auto_tile_for_size(width16, height16)

            dtype_mode = str(w_dtype_mode.value)
            target_dtype = _dtype_from_mode(dtype_mode)
            if target_dtype == torch.bfloat16 and not _bf16_is_supported_cuda():
                print("⚠️ BF16 似乎不支持，自动降级为 FP16")
                target_dtype = torch.float16
            dtype_apply_enabled = bool(w_dtype_apply.value)

            svh_enabled = bool(w_svh_enable.value)
            svh_seed_mode = str(w_svh_seed_mode.value)

            print("=" * 50)
            print(f"🎯 批量生成: {batch_n} 张")
            print(f"   尺寸: {width16}×{height16} | 步数: {steps} | CFG: {cfg}")
            print(f"   allocator: {_get_allocator_conf() or '(not set)'}")
            print(f"   dtype_mode={dtype_mode} | apply={dtype_apply_enabled}")
            print("=" * 50)

            if torch.cuda.is_available():
                try: torch.cuda.reset_peak_memory_stats()
                except Exception: pass
                _print_cuda_mem_line("before")

            _prepare_full_gpu(p, attn_slice)
            print(f"✅ GPU 准备完成，注意力切片: {('max' if attn_slice=='auto' else attn_slice)}")

            _maybe_cast_pipe_modules(p, target_dtype=target_dtype, enable=dtype_apply_enabled)

            sdpa_ctx = _enter_sdpa_efficient()
            if sdpa_ctx is not None:
                print("✅ SDPA: EFFICIENT_ATTENTION 优先 已启用")
            else:
                print("⚠️ SDPA: 未能显式启用（将按默认实现/切片运行）")

            ok_int8, msg_int8 = _apply_int8_matmul_to_transformer(p, int8_enable)
            print(("✅ " if ok_int8 else "⚠️ ") + msg_int8)

            # LoRA：批量开始前应用一次
            slot_files = [w_lora1.value or None, w_lora2.value or None, w_lora3.value or None]
            slot_weights = [float(w_lora1_w.value), float(w_lora2_w.value), float(w_lora3_w.value)]

            try:
                _apply_lora_slots_to_pipe(
                    p,
                    slot_files=slot_files,
                    slot_weights=slot_weights,
                    lora_folder=w_lora_dir.value.strip(),
                )
                active = [(i+1, slot_files[i], slot_weights[i]) for i in range(3) if slot_files[i]]
                if active:
                    print("🎭 LoRA 已应用槽位:")
                    for s, fn, wv in active:
                        print(f"   - 槽位{s}: {fn} | strength={wv}")
                else:
                    print("🎭 LoRA: 未加载（保持基线显存）")
            except Exception as e:
                print("⚠️ LoRA 应用失败（已忽略，不影响无LoRA推理）:", e)

            lora_slots_meta = {
                "slot1_file": slot_files[0] or "",
                "slot1_weight": slot_weights[0],
                "slot2_file": slot_files[1] or "",
                "slot2_weight": slot_weights[1],
                "slot3_file": slot_files[2] or "",
                "slot3_weight": slot_weights[2],
                "lora_folder": w_lora_dir.value.strip(),
            }

            for i in range(batch_n):
                latent = None
                try:
                    real_seed = int(np.random.randint(0, 2**32 - 1)) if seed_in == -1 else int(seed_in + i)
                    svh_seed = int(w_svh_seed.value) if (svh_seed_mode == "custom") else int(real_seed)

                    svh_meta = {
                        "enabled": svh_enabled,
                        "randomize_percent": float(w_svh_randomize_percent.value),
                        "strength": float(w_svh_strength.value),
                        "noise_insert": str(w_svh_noise_insert.value),
                        "steps_switchover_percent": float(w_svh_steps_switchover_percent.value),
                        "seed_mode": str(w_svh_seed_mode.value),
                        "seed": int(svh_seed),
                        "mask_starts_at": str(w_svh_mask_starts_at.value),
                        "mask_percent": float(w_svh_mask_percent.value),
                    }

                    print("-" * 50)
                    print(f"🖼️ [{i+1}/{batch_n}] seed={real_seed}")

                    t0 = time.time()
                    with torch.inference_mode():
                        te_dtype = target_dtype if dtype_apply_enabled else torch.float16

                        embeds_kwargs = _encode_prompt_embeds_then_drop_te(
                            p, prompt_text=prompt, negative_text=negative, cfg=cfg, te_dtype=te_dtype
                        )

                        embeds_kwargs = _svh_apply_to_embeds(
                            embeds_kwargs,
                            enable=svh_enabled,
                            randomize_percent=float(w_svh_randomize_percent.value),
                            strength=float(w_svh_strength.value),
                            noise_insert=str(w_svh_noise_insert.value),
                            steps_switchover_percent=float(w_svh_steps_switchover_percent.value),
                            seed=int(svh_seed),
                            mask_starts_at=str(w_svh_mask_starts_at.value),
                            mask_percent=float(w_svh_mask_percent.value),
                            log_to_console=bool(w_svh_log_to_console.value),
                        )

                        if torch.cuda.is_available():
                            runtime_fragmentation_cleanup(synchronize=True, reset_peak=True)

                        gen = torch.Generator(device="cpu").manual_seed(real_seed)
                        result = p(
                            prompt=None,
                            **embeds_kwargs,
                            height=height16,
                            width=width16,
                            num_inference_steps=steps,
                            guidance_scale=cfg,
                            generator=gen,
                            output_type="latent",
                        )
                        latent = result.images[0]

                    img = _decode_latent_with_pipe_vae(p, latent, scaling_factor=scaling, tiled=tiled, tile_size=tile_size)
                    display(img)

                    metadata = _build_generation_metadata(
                        prompt=prompt,
                        negative=negative,
                        seed=real_seed,
                        steps=steps,
                        cfg=cfg,
                        width=width16,
                        height=height16,
                        ratio_name=w_ratio.value,
                        svh_meta=svh_meta,
                        lora_slots=lora_slots_meta,
                        pipe=p,
                    )

                    ts = datetime.now().strftime("%H%M%S")
                    base_name = f"{real_seed}_{width16}x{height16}_{steps}s_{ts}"

                    if save_latent:
                        latent_path = os.path.join(latent_dir, f"latent_{base_name}.npz")
                        if _save_latent_with_metadata(latent, metadata, latent_path):
                            print(f"💾 Latent: {latent_path}")
                            verify_meta = _read_metadata_from_latent(latent_path)
                            if verify_meta:
                                print(f"   ✓ 元数据验证成功 (seed={verify_meta.get('seed')})")

                    if save_png:
                        png_path = os.path.join(png_dir, f"img_{base_name}.png")
                        if _save_png_with_metadata(img, metadata, png_path):
                            print(f"💾 PNG: {png_path}")
                            verify_meta = _read_metadata_from_png(png_path)
                            if verify_meta:
                                print(f"   ✓ 元数据验证成功 (seed={verify_meta.get('seed')})")

                    dt = time.time() - t0
                    print(f"⏱️ 单张耗时: {dt:.2f} 秒")

                except Exception:
                    traceback.print_exc()
                finally:
                    try:
                        del latent
                    except Exception:
                        pass
                    if torch.cuda.is_available():
                        try: torch.cuda.empty_cache()
                        except Exception: pass
                    gc.collect()

            print("=" * 50)
            print("✨ 批量完成！")

        except Exception:
            traceback.print_exc()
        finally:
            _exit_sdpa(sdpa_ctx)
            if torch.cuda.is_available():
                try: torch.cuda.empty_cache()
                except Exception: pass
            gc.collect()

def _on_clear(_):
    out.clear_output()

# ==================== UI 布局工具 ====================
def make_section(title_html, *children):
    return widgets.VBox([
        widgets.HTML(f'<div class="zimg-section-title">{title_html}</div>'),
        *children,
    ], layout=widgets.Layout(
        padding="16px",
        margin="8px 0",
        border="1px solid #334155",
        border_radius="12px",
        background="#1e293b",
    ))

def make_collapsible_section(title_html, content_widget: widgets.Widget, open_default: bool = True):
    acc = widgets.Accordion(children=[content_widget])
    acc.set_title(0, title_html)
    acc.selected_index = 0 if open_default else None
    acc.layout = widgets.Layout(margin="8px 0")
    try:
        acc.style = {"_dom_classes": ["zimg-accordion"]}
    except Exception:
        pass
    return acc

# ==================== Sections ====================
prompt_section = make_section(
    "📝 提示词",
    widgets.HTML('<div class="zimg-label">正向提示词</div>'),
    prompt_box,
    widgets.HTML('<div class="zimg-label" style="margin-top:12px;">负面提示词（CFG≤1无效）</div>'),
    neg_box,

    # ✅ 新增：位置放在负向提示词下面
    widgets.HTML('<div class="zimg-label" style="margin-top:12px;">📥 一键复制/回填 PNG 元数据</div>'),
    w_png_meta_path,
    widgets.HBox([btn_load_meta, btn_copy_meta_json], layout=widgets.Layout(gap="12px")),
    meta_status,
)

size_section = make_section(
    "📐 图像尺寸",
    widgets.HTML('<div class="zimg-label">预设比例（选择后保持比例联动，或选\"自定义\"自由调整）</div>'),
    w_ratio,
    w_width,
    w_height,
    w_size_info,
)

gen_section = make_section(
    "⚙️ 生成参数",
    w_steps,
    w_cfg,
    w_seed,
    widgets.HTML('<div class="zimg-info">💡 种子 -1 表示随机生成；批量时若非 -1，则使用 seed+i</div>'),
)

opt_section_content = make_section(
    "🚀 显存优化",
    w_attn,
    w_int8,
    w_dtype_mode,
    w_dtype_apply,
    dtype_hint,
    widgets.HTML('<div class="zimg-info">💡 注意力切片选 auto 时将按 max 执行（更省显存）</div>'),
)
opt_section = make_collapsible_section("🚀 显存优化", opt_section_content, open_default=True)

vae_section_content = make_section(
    "🖼️ VAE 解码",
    w_scaling,
    w_tiled,
    w_tile,
    widgets.HTML("<div class='zimg-info'>💡 生成大图（≥1536px）建议开启分块解码；tile=256/384/512 更稳</div>"),
)
vae_section = make_collapsible_section("🖼️ VAE 解码", vae_section_content, open_default=False)

save_section_content = make_section(
    "💾 保存设置",
    widgets.HBox([w_save_latent, w_save_png]),
    w_latent_dir,
    w_png_dir,
)
save_section = make_collapsible_section("💾 保存设置", save_section_content, open_default=False)

lora_section_content = make_section(
    "🎭 LoRA（3槽位·推理前自动应用）",
    widgets.HTML('''
    <div style="color:#64748b;font-size:12px;padding:8px;background:#0f172a;border-radius:8px;">
        📋 槽位显示 LoRA 文件名（避免误用）；推理前会按槽位状态自动加载/卸载并设置权重（若pipe支持）。
    </div>
    '''),
    lora_slots_ui,
)
lora_section = make_collapsible_section("🎭 LoRA（3槽位·推理前自动应用）", lora_section_content, open_default=False)

svh_section_content = make_section(
    "🧬 SeedVarianceEnhancer",
    widgets.HTML("""
    <div style="color:#64748b;font-size:12px;padding:8px;background:#0f172a;border-radius:8px;">
      说明：此处在 <b>TE 编码后</b> 对 <code>prompt_embeds</code>（以及 CFG&gt;1 时的 <code>negative_prompt_embeds</code>）做噪声注入，增强多样性。<br>
      ⚠️ Colab pipeline 模式不支持 ComfyUI conditioning 的"按步 begin/end 切换"，因此 begin/end 会按 all steps 等价处理（参数仍保留以便记录/迁移）。
    </div>
    """),
    w_svh_enable,
    w_svh_randomize_percent,
    w_svh_strength,
    w_svh_noise_insert,
    w_svh_steps_switchover_percent,
    w_svh_mask_starts_at,
    w_svh_mask_percent,
    w_svh_seed_mode,
    w_svh_seed,
    w_svh_log_to_console,
)
svh_section = make_collapsible_section("🧬 SeedVarianceEnhancer（多样性/种子扰动）", svh_section_content, open_default=False)

# ==================== 按钮区：在生成按钮旁加入"每次张数" ====================
btn_run.on_click(_on_run)
btn_clear.on_click(_on_clear)

button_section = widgets.HBox(
    [btn_run, w_batch, btn_clear],
    layout=widgets.Layout(justify_content="center", margin="16px 0", gap="16px", align_items="center")
)

main_panel = widgets.VBox([
    header_html,
    allocator_html,
    prompt_section,
    size_section,
    gen_section,
    opt_section,
    svh_section,
    vae_section,
    save_section,
    lora_section,
    button_section,
    widgets.HTML('<div class="zimg-section-title" style="margin-top:16px;">📺 输出区域</div>'),
    out,
], layout=widgets.Layout(
    width="800px",
    padding="20px",
    border_radius="16px",
    background="#0f172a",
))

display(main_panel)

In [ ]:
# @title 🧩 Latent 解码模块(使用不同VAE解码，不懂就不用)
import os, re, gc, json, zipfile, tarfile, hashlib, traceback
import numpy as np
import torch
from PIL import Image, PngImagePlugin
from IPython.display import display
import ipywidgets as widgets
from datetime import datetime

OUT_DIR_DEFAULT = "/content/drive/MyDrive/Z-image/VAE-image"
os.makedirs(OUT_DIR_DEFAULT, exist_ok=True)

# -------------------------
# utils
# -------------------------
def _hard_cleanup():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        try:
            torch.cuda.ipc_collect()
        except Exception:
            pass

def _safe_str(v, max_len=8000):
    try:
        s = json.dumps(v, ensure_ascii=False) if isinstance(v, (dict, list, tuple)) else str(v)
    except Exception:
        s = str(v)
    if max_len is not None and len(s) > max_len:
        s = s[:max_len] + "...(trunc)"
    return s

def _sha256(path):
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(1024 * 1024), b""):
            h.update(chunk)
    return h.hexdigest()

def _ensure_parent_dir(fp: str):
    d = os.path.dirname(fp)
    if d:
        os.makedirs(d, exist_ok=True)

def _print_gpu_mem():
    if torch.cuda.is_available():
        alloc = torch.cuda.memory_allocated() / (1024**3)
        resv = torch.cuda.memory_reserved() / (1024**3)
        print(f"💾 GPU allocated={alloc:.2f} GB | reserved={resv:.2f} GB")
    else:
        print("💾 CUDA not available (CPU runtime)")

# 让日志既进 out，又进 stdout（你在哪看都能看到）
def _log(msg: str):
    print(msg)

# -------------------------
# latent load (npz / npy / dict-wrapped)
# -------------------------
def _unwrap_object_array(x, max_depth=10):
    cur = x
    for _ in range(max_depth):
        if isinstance(cur, np.ndarray) and cur.dtype == np.object_:
            if cur.size == 1:
                cur = cur.reshape(-1)[0]
                continue
            return cur
        if isinstance(cur, np.generic):
            try:
                cur2 = cur.item()
                if cur2 is cur:
                    return cur
                cur = cur2
                continue
            except Exception:
                return cur
        return cur
    return cur

def _to_numeric_ndarray(x, prefer_dtype=np.float32):
    x = _unwrap_object_array(x)

    if isinstance(x, np.ndarray) and x.dtype != np.object_:
        if prefer_dtype is not None and x.dtype != prefer_dtype:
            if np.issubdtype(x.dtype, np.floating) or np.issubdtype(x.dtype, np.integer):
                x = x.astype(prefer_dtype, copy=False)
        return x

    if torch.is_tensor(x):
        x = x.detach().cpu().numpy()
        if prefer_dtype is not None and x.dtype != prefer_dtype:
            if np.issubdtype(x.dtype, np.floating) or np.issubdtype(x.dtype, np.integer):
                x = x.astype(prefer_dtype, copy=False)
        return x

    if isinstance(x, (list, tuple)):
        arr = np.array(x)
        arr = _unwrap_object_array(arr)
        if isinstance(arr, np.ndarray) and arr.dtype != np.object_:
            if prefer_dtype is not None and arr.dtype != prefer_dtype:
                if np.issubdtype(arr.dtype, np.floating) or np.issubdtype(arr.dtype, np.integer):
                    arr = arr.astype(prefer_dtype, copy=False)
            return arr

    raise TypeError(
        "latent 无法转换为数值 ndarray。\n"
        f"- type={type(x)}\n"
        f"- repr={_safe_str(x, max_len=600)}"
    )

def _merge_meta_from_dict(d: dict):
    meta = {}
    for k, v in d.items():
        if k == "latent":
            continue
        try:
            if isinstance(v, np.generic):
                v = v.item()
        except Exception:
            pass
        meta[str(k)] = v
    return meta

def load_latent_any(path: str):
    if not os.path.exists(path):
        raise RuntimeError("latent 路径不存在: " + path)

    lower = path.lower()

    if lower.endswith(".npy"):
        raw = np.load(path, allow_pickle=True)
        raw = _unwrap_object_array(raw)
        if isinstance(raw, dict):
            if "latent" not in raw:
                raise RuntimeError("该 .npy(dict) 内没有 'latent' 键，可用 keys=" + ", ".join(map(str, raw.keys())))
            latent = _to_numeric_ndarray(raw["latent"], prefer_dtype=np.float32)
            meta = _merge_meta_from_dict(raw)
            return latent, meta
        latent = _to_numeric_ndarray(raw, prefer_dtype=np.float32)
        return latent, {}

    data = np.load(path, allow_pickle=True)
    data = _unwrap_object_array(data)

    if isinstance(data, dict):
        if "latent" not in data:
            raise RuntimeError("该文件(dict)内没有 'latent' 键，可用 keys=" + ", ".join(map(str, data.keys())))
        latent = _to_numeric_ndarray(data["latent"], prefer_dtype=np.float32)
        meta = _merge_meta_from_dict(data)
        return latent, meta

    if isinstance(data, np.ndarray):
        latent = _to_numeric_ndarray(data, prefer_dtype=np.float32)
        return latent, {}

    if not hasattr(data, "files"):
        raise RuntimeError(f"无法解析该文件为 npz/npy/dict：np.load 类型={type(data)}")

    if "latent" not in data.files:
        raise RuntimeError(f"npz 内找不到 key=latent，可用 keys={data.files}")

    latent = _to_numeric_ndarray(data["latent"], prefer_dtype=np.float32)

    meta = {}
    if "metadata_json" in data.files:
        try:
            meta_json = data["metadata_json"].item()
            meta = json.loads(meta_json) if isinstance(meta_json, str) else json.loads(str(meta_json))
        except Exception:
            meta = {"metadata_json_raw": _safe_str(data["metadata_json"].item())}

    for k in data.files:
        if k.startswith("meta_"):
            key = k[5:]
            vv = data[k]
            try:
                vv = vv.item() if hasattr(vv, "item") else vv
            except Exception:
                pass
            meta[key] = vv

    return latent, meta

# -------------------------
# PNG write meta
# -------------------------
def save_png_with_all_meta(path: str, img: Image.Image, meta: dict):
    pnginfo = PngImagePlugin.PngInfo()

    prompt = (meta.get("prompt", "") or "")
    negative = (meta.get("negative_prompt", "") or "")
    parameters = prompt
    if negative.strip():
        parameters += "\nNegative prompt: " + negative
    pnginfo.add_text("parameters", parameters)

    pnginfo.add_text("zimg_meta_json", json.dumps(meta, ensure_ascii=False, sort_keys=True))

    for k, v in meta.items():
        try:
            pnginfo.add_text(str(k), _safe_str(v, max_len=8000))
        except Exception:
            pass

    _ensure_parent_dir(path)
    img.save(path, pnginfo=pnginfo)

# -------------------------
# VAE download / resolve / load
# -------------------------
def download_url_to_file(url: str, dst_file: str):
    url = (url or "").strip()
    if not re.match(r"^https?://", url, re.I):
        raise RuntimeError("URL 无效: " + url)
    dst_file = (dst_file or "").strip()
    if not dst_file:
        raise RuntimeError("请填写下载目标文件路径（建议放Drive）")
    _ensure_parent_dir(dst_file)

    if os.path.exists(dst_file) and os.path.getsize(dst_file) > 0:
        _log("ℹ️ 已存在，跳过下载: " + dst_file)
        return dst_file

    import urllib.request
    _log("🌐 downloading: " + url)
    urllib.request.urlretrieve(url, dst_file)
    _log("✅ downloaded: " + dst_file + " sha256:" + _sha256(dst_file)[:16] + "...")
    return dst_file

def extract_archive_if_needed(src_path: str, dst_dir: str):
    lower = src_path.lower()
    os.makedirs(dst_dir, exist_ok=True)

    if lower.endswith(".zip"):
        _log("📦 extracting zip -> " + dst_dir)
        with zipfile.ZipFile(src_path, "r") as z:
            z.extractall(dst_dir)
        return dst_dir

    if lower.endswith(".tar") or lower.endswith(".tar.gz") or lower.endswith(".tgz"):
        _log("📦 extracting tar -> " + dst_dir)
        with tarfile.open(src_path, "r:*") as t:
            t.extractall(dst_dir)
        return dst_dir

    return src_path if os.path.isdir(src_path) else os.path.dirname(src_path)

def resolve_vae_dir(root_dir: str):
    if os.path.isdir(root_dir) and os.path.exists(os.path.join(root_dir, "config.json")) and (
        os.path.exists(os.path.join(root_dir, "diffusion_pytorch_model.safetensors")) or
        os.path.exists(os.path.join(root_dir, "diffusion_pytorch_model.bin"))
    ):
        return root_dir

    cand = os.path.join(root_dir, "vae")
    if os.path.isdir(cand) and os.path.exists(os.path.join(cand, "config.json")):
        return cand

    for dirpath, dirnames, filenames in os.walk(root_dir):
        if "config.json" in filenames and (
            "diffusion_pytorch_model.safetensors" in filenames or "diffusion_pytorch_model.bin" in filenames
        ):
            if os.path.basename(dirpath).lower() == "vae":
                return dirpath

    raise RuntimeError(f"在 {root_dir} 下找不到可用的 VAE 目录（需要 config.json + 权重文件）")

def load_vae(vae_dir: str, device="cuda"):
    from diffusers import AutoencoderKL

    has_cuda = torch.cuda.is_available()
    if device == "cuda" and not has_cuda:
        _log("⚠️ 未检测到 CUDA，自动改用 CPU 解码")
        device = "cpu"

    dtype = torch.float16 if device == "cuda" else torch.float32
    _log(f"🔧 load_vae: device={device} dtype={dtype}")

    vae = AutoencoderKL.from_pretrained(vae_dir, torch_dtype=dtype, local_files_only=True)
    vae.eval()
    vae.to(device)
    return vae

# -------------------------
# decode
# -------------------------
def decode_latent_with_vae(vae, latent_np: np.ndarray, scaling_factor: float):
    if not isinstance(latent_np, np.ndarray):
        raise TypeError(f"latent 不是 ndarray：{type(latent_np)}")
    if latent_np.dtype == np.object_:
        raise TypeError(f"latent dtype=object（仍未解包成功）。shape={latent_np.shape}")

    lat = torch.from_numpy(latent_np)
    if lat.ndim == 3:
        lat = lat.unsqueeze(0)
    if lat.ndim != 4:
        raise RuntimeError(f"latent 维度不对：{tuple(lat.shape)}（期望 NCHW）")

    device = next(vae.parameters()).device
    dtype = torch.float16 if device.type == "cuda" else torch.float32
    lat = lat.to(device=device, dtype=dtype)

    with torch.no_grad():
        decoded = vae.decode(lat / float(scaling_factor)).sample

    img = (decoded[0] / 2 + 0.5).clamp(0, 1).detach().cpu().permute(1, 2, 0).float().numpy()
    return Image.fromarray((img * 255).astype(np.uint8))

# ======================================================
# UI
# ======================================================
_state = {"latent": None, "meta": None, "img": None, "vae_dir": None, "vae": None}

latent_path = widgets.Text(value="", description="latent", layout=widgets.Layout(width="99%"))

w_vae_load_path = widgets.Text(
    value="",
    description="加载路径",
    placeholder="本地 VAE 目录 / 模型根目录 / 压缩包文件(.zip/.tar/.tgz)",
    layout=widgets.Layout(width="99%"),
)
w_extract_dir = widgets.Text(value="/content/vae_extract", description="解压到", layout=widgets.Layout(width="99%"))

w_device = widgets.Dropdown(options=["cuda", "cpu"], value="cpu", description="device", layout=widgets.Layout(width="220px"))
w_scaling = widgets.FloatText(value=0.3611, description="scale", layout=widgets.Layout(width="220px"))

w_out_dir = widgets.Text(value=OUT_DIR_DEFAULT, description="out_dir", layout=widgets.Layout(width="99%"))
w_auto_name = widgets.Checkbox(value=True, description="自动命名输出文件", indent=False)
w_out_name = widgets.Text(value="vae_decode.png", description="文件名", layout=widgets.Layout(width="99%"))

btn_load_vae = widgets.Button(description="0) 加载VAE", button_style="")
btn_unload_vae = widgets.Button(description="卸载VAE", button_style="warning")
btn_decode = widgets.Button(description="1) 解码预览", button_style="")
btn_save = widgets.Button(description="2) 保存PNG（含全部meta）", button_style="success")
btn_clear = widgets.Button(description="清空输出", button_style="")

status = widgets.HTML("<div style='color:#64748b'>状态：就绪</div>")
out = widgets.Output(layout=widgets.Layout(width="980px", border="1px solid #334155", padding="10px"))

def _set_status(s: str):
    status.value = f"<div style='color:#64748b'><b>状态：</b>{s}</div>"

def _unload_current_vae():
    vae = _state.get("vae", None)
    _state["vae"] = None
    _state["vae_dir"] = None
    if vae is not None:
        try:
            vae.to("cpu")
        except Exception:
            pass
        del vae
    _hard_cleanup()

def on_unload_vae(_):
    out.clear_output()
    with out:
        _set_status("卸载VAE中…")
        print("[unload] clicked")
        _unload_current_vae()
        print("✅ VAE unloaded")
        _print_gpu_mem()
        _set_status("VAE已卸载")

def on_load_vae(_):
    # 双通道：stdout + out
    print("[load] clicked")  # stdout
    out.clear_output()
    with out:
        try:
            _set_status("加载VAE中…")
            print("[load] clicked (in out)")

            if _state.get("vae", None) is not None:
                print("ℹ️ 已有VAE，先卸载旧VAE…")
                _unload_current_vae()

            lp = (w_vae_load_path.value or "").strip()
            print("[load] load_path:", lp)
            if not lp:
                raise RuntimeError("加载路径为空：请填 VAE 目录/模型根/压缩包文件路径")
            if not os.path.exists(lp):
                raise RuntimeError("加载路径不存在: " + lp)

            root = lp
            if os.path.isfile(lp) and re.search(r"\.(zip|tar|tar\.gz|tgz)$", lp, re.I):
                ed = (w_extract_dir.value.strip() or "/content/vae_extract")
                print("[load] extracting to:", ed)
                root = extract_archive_if_needed(lp, ed)

            print("[load] resolve_vae_dir root:", root)
            vae_dir = resolve_vae_dir(root)
            _state["vae_dir"] = vae_dir
            print("📁 resolved vae_dir:", vae_dir)

            print("[load] load_vae… device:", w_device.value)
            vae = load_vae(vae_dir, device=w_device.value)
            _state["vae"] = vae
            print("✅ VAE loaded:", vae.__class__.__name__)
            _print_gpu_mem()
            _set_status("VAE已加载")
        except Exception:
            _set_status("加载VAE失败（见下方）")
            traceback.print_exc()
        finally:
            _hard_cleanup()

def on_decode(_):
    print("[decode] clicked")
    out.clear_output()
    with out:
        try:
            _set_status("解码中…")
            if _state.get("vae", None) is None:
                raise RuntimeError("请先点「0) 加载VAE」")

            lp = latent_path.value.strip()
            print("[decode] latent_path:", lp)
            latent, meta = load_latent_any(lp)
            _state["latent"] = latent
            _state["meta"] = meta

            if "vae_scaling_factor" in meta:
                try:
                    w_scaling.value = float(meta["vae_scaling_factor"])
                except Exception:
                    pass

            img = decode_latent_with_vae(_state["vae"], latent, scaling_factor=float(w_scaling.value))
            _state["img"] = img
            display(img)
            print("✅ decoded | img.size:", img.size)
            print("latent:", latent.shape, latent.dtype)
            print("meta keys:", len(meta))
            _set_status("解码完成")
        except Exception:
            _set_status("解码失败（见下方）")
            traceback.print_exc()
        finally:
            _hard_cleanup()

def on_save(_):
    print("[save] clicked")
    out.clear_output()
    with out:
        try:
            _set_status("保存中…")
            if _state.get("img", None) is None or _state.get("meta", None) is None:
                raise RuntimeError("请先点「1) 解码预览」")

            out_dir = (w_out_dir.value or OUT_DIR_DEFAULT).strip()
            os.makedirs(out_dir, exist_ok=True)

            meta = dict(_state["meta"])
            meta["vae_decode"] = {
                "vae_load_path": (w_vae_load_path.value or "").strip(),
                "vae_dir_resolved": _state.get("vae_dir"),
                "scaling_factor": float(w_scaling.value),
                "device": w_device.value,
                "decoded_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "latent_path": latent_path.value.strip(),
            }

            if bool(w_auto_name.value):
                base = os.path.splitext(os.path.basename(latent_path.value.strip()))[0]
                out_name = f"vae_decode_{base}.png"
            else:
                out_name = (w_out_name.value or "vae_decode.png").strip()

            out_path = os.path.join(out_dir, out_name)
            save_png_with_all_meta(out_path, _state["img"], meta)
            print("💾 saved:", out_path)
            _set_status("保存完成")
        except Exception:
            _set_status("保存失败（见下方）")
            traceback.print_exc()
        finally:
            _hard_cleanup()

def on_clear(_):
    out.clear_output()
    _set_status("就绪")

btn_load_vae.on_click(on_load_vae)
btn_unload_vae.on_click(on_unload_vae)
btn_decode.on_click(on_decode)
btn_save.on_click(on_save)
btn_clear.on_click(on_clear)

ui = widgets.VBox(
    [
        widgets.HTML("<b>CPU/GPU 通用 Latent 解码</b><div style='color:#64748b;font-size:12px'>"
                     "已加入双通道日志：按钮回调一定会在 stdout 和 out 区显示 clicked，避免“看起来没反应”。</div>"),
        widgets.HTML("<hr><b>① latent</b>"),
        latent_path,
        widgets.HTML("<hr><b>② VAE 加载</b>"),
        w_vae_load_path,
        w_extract_dir,
        widgets.HBox([w_device, w_scaling]),
        widgets.HBox([btn_load_vae, btn_unload_vae, btn_decode, btn_save, btn_clear]),
        status,
        widgets.HTML("<hr><b>③ 输出</b>"),
        w_out_dir,
        widgets.HBox([w_auto_name]),
        w_out_name,
        out,
    ],
    layout=widgets.Layout(width="980px"),
)

display(ui)